In [ ]:
pip install pandas networkx gensim node2vec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch

In [ ]:
!pip install tensorflow_ranking


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.


In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
!pip install node2vec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torc

In [ ]:
import pandas as pd
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from node2vec import Node2Vec
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

#load the data
data = pd.read_csv("yoochoose1.csv")

#convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

#sort the data by session_id and timestamp
data.sort_values(by=['session_id', 'timestamp'], inplace=True)

#encode session_id and item_id columns
encoder_session = LabelEncoder()
encoder_item = LabelEncoder()
data['session_id'] = encoder_session.fit_transform(data['session_id'])
data['item_id'] = encoder_item.fit_transform(data['item_id'])

#construct graph
graph = nx.Graph()
for session_id, group in data.groupby('session_id'):
    items = list(group['item_id'])
    for i in range(len(items) - 1):
        graph.add_edge(items[i], items[i + 1])

# Generate walks(node2vec)
node2vec = Node2Vec(graph, dimensions=128, walk_length=10, num_walks=20, workers=4)

#train the model
model = node2vec.fit(window=10, min_count=1, batch_words=4)

#get embeddings
embeddings = {}
for node in graph.nodes():
    try:
        embeddings[node] = model.wv[node]
    except KeyError:
        # if node is not present in the model vocabulary then assignd a random embedding
        embeddings[node] = np.random.uniform(-1, 1, model.vector_size)

#recommendation
def recommend(session_id, top_n=5):
    session_items = data[data['session_id'] == session_id]['item_id'].unique()
    session_embedding = np.mean([embeddings[item] for item in session_items if item in embeddings], axis=0)

    scores = {}
    for item in graph.nodes():
        if item not in session_items and item in embeddings:
            item_embedding = embeddings[item]
            scores[item] = np.dot(session_embedding, item_embedding)

    top_items = sorted(scores, key=scores.get, reverse=True)[:top_n]
    return [encoder_item.inverse_transform([item])[0] for item in top_items]

#test
test_session_id = 5
recommended_items = recommend(test_session_id)
print(f"Recommended items for session {test_session_id}: {recommended_items}")

#split data into train and test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

#model defination
input_dim = len(encoder_item.classes_)
embedding_dim = 128
input_layer = Input(shape=(1,))
embedding_layer = Dense(embedding_dim, activation='relu')(input_layer)
output_layer = Dense(input_dim, activation='softmax')(embedding_layer)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy())

#training data
X_train = train_data['item_id']
y_train = train_data['item_id']

#testing data
X_test = test_data['item_id']
y_test = test_data['item_id']

#reshape the input data
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

#model training
model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test), verbose=2)

#model evaluation
predicted_embeddings = model.predict(X_test)
predicted_items = [np.argmax(embedding) for embedding in predicted_embeddings]
actual_items = y_test.tolist()

#MRR calculation
ranks = []
for actual, predicted in zip(actual_items, predicted_items):
    # Check if the predicted item has embeddings
    if predicted < len(predicted_embeddings):
        rank = np.where(np.argsort(-predicted_embeddings[predicted]))[0][actual]
        ranks.append(rank)
mrr = np.mean([1.0 / (rank + 1) for rank in ranks])
print(f"Mean Reciprocal Rank (MRR): {mrr}")

#Recall@5 calculation
recall_at_5 = sum([1 for rank in ranks if rank < 5]) / len(ranks)
print(f"Recall@5: {recall_at_5}")


Computing transition probabilities:   0%|          | 0/577 [00:00<?, ?it/s]

Recommended items for session 5: [214543492, 214819526, 214539039, 214584907, 214553865]
Epoch 1/10
27/27 - 1s - loss: 36.2930 - val_loss: 28.3102 - 677ms/epoch - 25ms/step
Epoch 2/10
27/27 - 0s - loss: 14.7870 - val_loss: 24.5028 - 120ms/epoch - 4ms/step
Epoch 3/10
27/27 - 0s - loss: 9.4034 - val_loss: 25.1723 - 143ms/epoch - 5ms/step
Epoch 4/10
27/27 - 0s - loss: 7.9335 - val_loss: 26.2883 - 123ms/epoch - 5ms/step
Epoch 5/10
27/27 - 0s - loss: 7.5581 - val_loss: 27.7584 - 126ms/epoch - 5ms/step
Epoch 6/10
27/27 - 0s - loss: 7.2896 - val_loss: 29.0161 - 112ms/epoch - 4ms/step
Epoch 7/10
27/27 - 0s - loss: 7.1849 - val_loss: 30.3545 - 122ms/epoch - 5ms/step
Epoch 8/10
27/27 - 0s - loss: 7.1220 - val_loss: 31.5047 - 112ms/epoch - 4ms/step
Epoch 9/10
27/27 - 0s - loss: 7.1099 - val_loss: 32.1342 - 110ms/epoch - 4ms/step
Epoch 10/10
27/27 - 0s - loss: 7.1082 - val_loss: 32.9404 - 118ms/epoch - 4ms/step
7/7 [==============================] - 0s 2ms/step
Mean Reciprocal Rank (MRR): 0.131893